In [26]:
import pandas as pd
import numpy as np

import pickle
from tqdm.notebook import trange, tqdm

import re    # RegEx for removing non-letter characters
import nltk  #natural language processing
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliorenteria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
data = pd.read_csv('./data/vaccineTourism.csv')
print(data.shape)
data.head()

(770, 2)


,text,date
0,the tourism authority of kiribati (tak) welcom...,2021-05-31 05:09:50
1,i’m in canada. it’s a race to protect agains...,2021-05-31 04:48:36
2,‘like a dream’: latin americans head to u.s. f...,2021-05-31 04:28:08
3,"i love this. i know it's not equitable, but th...",2021-05-31 04:15:47
4,tourism agency warns against \n\n tourism de...,2021-05-31 04:15:31


In [16]:
# Function to format, tokenize and remove stopwords from tweets.
def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove tweeter users and hashtags ( @xxx, #xxx )
    text = re.sub(r"[@#]\w+", " ", text)
    # remove https
    text = re.sub(r"[(http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&\/\/=]*)", " ", text)
    #text = " ".join(filter(lambda w: w.find("https://") == -1, text.split(" ")))
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    #remove numbers
    text = re.sub(r"[0-9]*", "", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

In [17]:
# Testing tweet_to_words Function
sel = 103
print("\nORIGINAL TWEET ->\n", data['text'][sel])
print("\nPROCESSED TWEET ->", tweet_to_words(data['text'][sel]))



ORIGINAL TWEET ->
   once again money buys happiness or at least life in this case! us should be ashamed, but hey it's good for…  

PROCESSED TWEET -> ['money', 'buy', 'happi', 'least', 'life', 'case', 'us', 'asham', 'hey', 'good']


In [18]:
# Apply tokenization to each tweet and store as X
X = []
for i in trange(len(data['text'])):
    X.append(tweet_to_words(data['text'][i]))

In [23]:
# load the model from disk
filename = 'XGBoost_tf_Covid.sav'
loaded_model = pickle.load(open('./saved_models/'+filename, 'rb'))
if loaded_model != False:
    print('"{}" model loaded correctly'.format(filename))
else:
    print('error loading model')
loaded_model

"XGBoost_tf_Covid.sav" model loaded correctly


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=1.0, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=13, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [24]:
loaded_model.predict(X)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (770,) + inhomogeneous part.